# Parallel Query Execution

This notebook shows how to fire up concurrent queries in Spark. This may be useful to get a better overall throughput in cases that multiple outputs need to be generated.

We will simply reuse the weather example and fire up two concurrent queries. Although they will generate the very same result, it is still interesting to see that even the intermediate cache will only be built once.

## 0. Prerequisites

Running multiple queries in parallel requires some configuration of Spark. Spark can always accept multiple queries, but per default it will process those in a *FIFO* fashion. This means that one query is processed after the other. But Spark also supports a real parallel query execution using a different task scheduler.

You need to configure the following values:

1. Create a schduler configuration file `fairscheduler.xml` (contents see below)
2. Set Spark config `spark.scheduler.mode` to `FAIR`
3. Set Spark config `spark.scheduler.allocation.file` to the location of the `fairscheduler.xml` file

Unfortunately these values need to configures __before the Spark session is created__.

The `fairscheduler.xml` should look as follows:
```xml
<?xml version="1.0"?>
<allocations>
  <pool name="fair">
    <schedulingMode>FAIR</schedulingMode>
    <weight>1</weight>
    <minShare>2</minShare>
  </pool>
</allocations>
```

## 1. Read in all years

Now we read in all years by creating a union. We also add the year as a logical partition column, this will be used later.

In [3]:
from pyspark.sql.functions import *

In [4]:
storageLocation = "s3://dimajix-training/data/weather"

In [5]:
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

## 2. Extract Information

The raw data is not exactly nice to work with, so we need to extract the relevant information by using appropriate substr operations.

In [6]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)

## 3. Read in Station Metadata

Fortunately station metadata is stored as CSV, so we can directly read that using Sparks `spark.read.csv` mechanisum. The data can be found at `storageLocation + '/isd-history'`.

In [7]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

## 4. Join and cache data

Now we need to join the meta data with the measurements. We will cache the result

In [8]:
joined_data = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
joined_data.cache()

DataFrame[year: int, usaf: string, wban: string, date: string, time: string, report_type: string, wind_direction: string, wind_direction_qual: string, wind_observation: string, wind_speed: double, wind_speed_qual: string, air_temperature: double, air_temperature_qual: string, USAF: string, WBAN: string, STATION NAME: string, CTRY: string, STATE: string, ICAO: string, LAT: string, LON: string, ELEV(M): string, BEGIN: string, END: string]

## 5. Perform Queries

Now comes the interesting part: We will perform multiple queries in parallel. We will make use of the Python `threading` module in order to start two concurrent queries. One query will aggregate min/max of temperature, while the other query will aggragte min/max of wind speed.

We will save both results to corresponding CSV files into HDFS.

### 5.1 Define Queries

First we create two Python functions which contain the two queries to be executed

In [15]:
def calc_temperature():
    df = joined_data
    result = df.groupBy(df.CTRY, df.year).agg(
            min(when(df.air_temperature_qual == lit(1), df.air_temperature)).alias('min_temp'),
            max(when(df.air_temperature_qual == lit(1), df.air_temperature)).alias('max_temp'),
        )
    result.write.csv("/user/hadoop/weather_min_max_temp")

In [16]:
def calc_wind_speed():
    df = joined_data
    result = df.groupBy(df.CTRY, df.year).agg(
            min(when(df.wind_speed_qual == lit(1), df.wind_speed)).alias('min_wind'),
            max(when(df.wind_speed_qual == lit(1), df.wind_speed)).alias('max_wind')
        )
    result.write.csv("/user/hadoop/weather_min_max_wind")


### 5.2 Run Queries

Now since we have the two functions, we import and use the Python `threading` module to run both queries in parallel. We also need to configure Spark to use the correct scheduler pool (in our case it is the `fair` pool).

In [17]:
# We need to set the thread local property "spark.scheduler.pool" to the correct pool defined in fairscheduler.xml
spark.sparkContext.setLocalProperty("spark.scheduler.pool", "fair")

In [ ]:
import threading

# First create threads
t1 = threading.Thread(target=calc_temperature)
t2 = threading.Thread(target=calc_wind_speed)

# Then start both threads (in the background)
t1.start()
t2.start()

# Finally wait until both threads have finished
t1.join()
t2.join()

### 5.3 Watch Query execution

Now you should open the Spark web interface and watch both queries being processed in parallel.

### 5.4 Inspect result

Finally you can inspect the results using for example Spark again (or HDFS command line tools).

In [ ]:
temp_df = spark.read.text("/user/hadoop/weather_min_max_temp")
temp_df.show()

In [ ]:
wind_df = spark.read.text("/user/hadoop/weather_min_max_wind")
wind_df.show()